In [1]:
# Python 샘플 코드 #
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
# 해당 코드에서 크롤링을 시도할 연도(yyyy00)
this_year = 200800

In [3]:
'''
국토교통부_아파트매매 실거래 상세 자료
활용기간 2023.04.12 까지
LAWD_CD = 11110  # 47290(경산) 지역코드 - 각 지역별 코드 행정표준코드관리시스템(www.code.go.kr)의 법정동코드 10자리 중 앞 5자리
DEAL_YMD = 200601  # 계약월 - 실거래 자료의 계약년월 6자리
numOfRows = 500  # 한 페이지 결과 수
pageNo = 1  # 페이지번호
'''
def get_apt_data(LAWD_CD, DEAL_YMD, serviceKey, numOfRows=10000):
    # 호출하려는 OpenAPI URL 정의
    data_url = "".join(
        ["http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?",
         "serviceKey=", serviceKey, "&LAWD_CD=", str(LAWD_CD), "&DEAL_YMD=", str(DEAL_YMD), "&numOfRows=",
         str(numOfRows)])

    # 정의된 OpenAPI URL을 호출
    data_response = requests.get(data_url)
    data_xml = BeautifulSoup(data_response.text, "xml")
    data_xml_row = data_xml.find_all("item")
    data_xml_row

    # XML에 포함된 항목들을 리스트로 저장
    list_items = ["거래금액", "건축년도", "년", "도로명", "도로명건물본번호코드", "도로명건물부번호코드",
                  "도로명시군구코드", "도로명일련번호코드", "도로명지상지하코드", "도로명코드",
                  "법정동", "법정동본번코드", "법정동부번코드", "법정동시군구코드", "법정동읍면동코드", "법정동지번코드",
                  "아파트", "월", "일", "일련번호", "전용면적", "지번", "지역코드", "층", "해제사유발생일", "해제여부"]

    # 리스트의 리스트 형태로 데이터프레임에 저장할 자료 만들기
    list_dataframe = []

    for n_obs in range(0, len(data_xml_row)):
        list_obs_values = []  # 동일한 관측치의 데이터는 모두 이 리스트에 저장된다
        for i in range(0, len(list_items)):
            # 특정 항목이 XML에 아예 존재하지 않는 경우가 있으므로 try 구문 사용
            try:
                list_obs_values.append(data_xml_row[n_obs].find(list_items[i]).text)
            except Exception as e:
                list_obs_values.append("")
                print("Error in " + str(n_obs) + "th obs, item: " + list_items[i] + ", " + str(e))
        list_dataframe.append(list_obs_values)  # 리스트의 리스트로 관측치 저장

    # 데이터프레임 형태로 변환
    df_apt = pd.DataFrame(list_dataframe, columns=list_items)
    return df_apt

In [4]:
# 함수 작동 여부 확인
serviceKey_Chun = "2ix6etGjWbVvOzBAIKHx7H6%2FAPHUZPbYUJM%2FUiVE8kqSUPIEEJp%2BhH9RBil1kfrFYeQSfJzfLDKR50JL0TKaGg%3D%3D"
df_tmp1 = get_apt_data(LAWD_CD=11110, DEAL_YMD=202102, serviceKey=serviceKey_Chun)


Error in 17th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 18th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 19th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 19th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 20th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 20th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 21th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 21th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 25th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 31th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 36th obs, item: 일련번호, 'NoneType' object has no attribute 'text'


In [5]:
# Import TXT file 법정동코드 전체자료 불러오기
df_lawd_cd = pd.read_csv('C:/Users/Kimyihwan/OneDrive - kaist.ac.kr/\\\
paper_prepare/data_raw/법정동코드 전체자료.txt', sep="\t", engine='python', encoding="CP949")
df_lawd_cd["LAWD_CD"] = df_lawd_cd["법정동코드"].divmod(100000)[0]
df_lawd_cd_nodup = df_lawd_cd.drop_duplicates(subset=['LAWD_CD']).copy()  # 중복값 제외
df_lawd_cd_nodup_exist = df_lawd_cd_nodup[df_lawd_cd_nodup["폐지여부"] == "존재"].copy()

In [6]:
total_lawd_cd = df_lawd_cd_nodup_exist["LAWD_CD"]
tmp_lawd_cd = df_lawd_cd_nodup_exist["LAWD_CD"][3:10]

In [7]:
# 국토교통부_아파트매매 실거래 상세 자료는 2008년 1월부터 자료가 존재한다.
# 그런데 2006년으로 입력해도 자료가 있다.

In [8]:

# 시작 시점과 종료 시점 입력
yyyymm_start = this_year + 7  # 시작 +-
yyyymm_end = this_year + 9  # 종료 년월

list_yyyymm = []
for n_yyyymm in range(yyyymm_start, yyyymm_end+1):
    if (divmod(n_yyyymm, 100)[1] != 0) & (divmod(n_yyyymm, 100)[1] < 13):
        list_yyyymm.append(n_yyyymm)

list_yyyymm.reverse()


# StopWatch: 코드 시작
time_this_code_start = datetime.now()
print("This code started at: " + str(time_this_code_start))

# DataFrame 구성을 위해 공백 자료를 하나 만든다.
df_dataset_null = get_apt_data(LAWD_CD=11120, DEAL_YMD=202102, serviceKey=serviceKey_Chun)

# 전국의 한달 자료 받는데 약 9분 소요
for yyyymm in list_yyyymm:
    df_dataset = df_dataset_null.copy()
    for lawd_cd in total_lawd_cd:
        df_apt = get_apt_data(LAWD_CD=lawd_cd, DEAL_YMD=yyyymm, serviceKey=serviceKey_Chun)
        # df_apt.to_pickle("./data_raw/df_apt_" + str(yyyymm) + "_" + str(lawd_cd) + ".pkl")
        df_dataset = pd.concat([df_dataset, df_apt])
    df_dataset.to_pickle("C:/Users/Kimyihwan/OneDrive - kaist.ac.kr/\\\
paper_prepare/data_raw/df_dataset_" + str(yyyymm) + ".pkl")

# StopWatch: 코드 종료
time_this_code_end = datetime.now()
print("This code finished at: " + str(time_this_code_end))
print("Elapsed (in this code): " + str(time_this_code_end - time_this_code_start))

This code started at: 2021-05-20 15:09:44.064648
Error in 6th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 16th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 19th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 4th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 6th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 43th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 9th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 14th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 66th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 68th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 15th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 17th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 22th obs, 

Error in 325th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 325th obs, item: 도로명건물본번호코드, 'NoneType' object has no attribute 'text'
Error in 325th obs, item: 도로명건물부번호코드, 'NoneType' object has no attribute 'text'
Error in 325th obs, item: 도로명시군구코드, 'NoneType' object has no attribute 'text'
Error in 325th obs, item: 도로명코드, 'NoneType' object has no attribute 'text'
Error in 345th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 345th obs, item: 도로명건물본번호코드, 'NoneType' object has no attribute 'text'
Error in 345th obs, item: 도로명건물부번호코드, 'NoneType' object has no attribute 'text'
Error in 345th obs, item: 도로명시군구코드, 'NoneType' object has no attribute 'text'
Error in 345th obs, item: 도로명코드, 'NoneType' object has no attribute 'text'
Error in 349th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 349th obs, item: 도로명건물본번호코드, 'NoneType' object has no attribute 'text'
Error in 349th obs, item: 도로명건물부번호코드, 'NoneType' object has no attribute 't

Error in 90th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 91th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 95th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 97th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 99th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 102th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 103th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 260th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 262th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 264th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 265th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 55th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 30th obs, item: 도로명지상지하코드, 'NoneType' object has 

Error in 56th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 58th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 1th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 28th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 123th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 126th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 133th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 136th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 137th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 144th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 145th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 147th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 151th obs, item: 도로명지상지하코드, 'NoneType' object ha

Error in 47th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 54th obs, item: 도로명일련번호코드, 'NoneType' object has no attribute 'text'
Error in 54th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 77th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 226th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 238th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 69th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 1327th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 37th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 38th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 40th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 41th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 46th obs, item: 도로명지상지하코드, 'NoneType' object has no

...

Error in 249th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 252th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 253th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 259th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 260th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 262th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 267th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 268th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 270th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 271th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 274th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 275th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 277th obs, item: 도로명지상지하코드, 'NoneType' obje

Error in 280th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 280th obs, item: 도로명건물본번호코드, 'NoneType' object has no attribute 'text'
Error in 280th obs, item: 도로명건물부번호코드, 'NoneType' object has no attribute 'text'
Error in 280th obs, item: 도로명시군구코드, 'NoneType' object has no attribute 'text'
Error in 280th obs, item: 도로명코드, 'NoneType' object has no attribute 'text'
Error in 281th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 281th obs, item: 도로명건물본번호코드, 'NoneType' object has no attribute 'text'
Error in 281th obs, item: 도로명건물부번호코드, 'NoneType' object has no attribute 'text'
Error in 281th obs, item: 도로명시군구코드, 'NoneType' object has no attribute 'text'
Error in 281th obs, item: 도로명코드, 'NoneType' object has no attribute 'text'
Error in 286th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 286th obs, item: 도로명건물본번호코드, 'NoneType' object has no attribute 'text'
Error in 286th obs, item: 도로명건물부번호코드, 'NoneType' object has no attribute 't

Error in 13th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 14th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 34th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 35th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 36th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 37th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 38th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 39th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 40th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 41th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 42th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 43th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 44th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 45th obs, item: 도로명, 'NoneType' object has 

Error in 255th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 258th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 260th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 266th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 267th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 269th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 0th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 1th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 2th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 3th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 4th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 5th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 6th obs, item: 도로명지상지하코드, 'NoneType' object has no attr

Error in 19th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 69th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 71th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 74th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 75th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 77th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 78th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 81th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 82th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 345th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 346th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 111th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 54th obs, item: 도로명지상지하코드, 'NoneType' object has no 

In [9]:
# Check the data

data1 = pd.read_pickle('C:/Users/Kimyihwan/OneDrive - kaist.ac.kr/\\\
paper_prepare/data_raw/df_dataset_'+str(yyyymm_start)+'.pkl')

data2 = pd.read_pickle('C:/Users/Kimyihwan/OneDrive - kaist.ac.kr/\\\
paper_prepare/data_raw/df_dataset_'+str(yyyymm_start + 1)+'.pkl')

data3 = pd.read_pickle('C:/Users/Kimyihwan/OneDrive - kaist.ac.kr/\\\
paper_prepare/data_raw/df_dataset_'+str(yyyymm_start + 2)+'.pkl')

In [10]:
data1

,거래금액,건축년도,년,도로명,도로명건물본번호코드,도로명건물부번호코드,도로명시군구코드,도로명일련번호코드,도로명지상지하코드,도로명코드,...,아파트,월,일,일련번호,전용면적,지번,지역코드,층,해제사유발생일,해제여부
0,"59,000",2000,2008,자하문로33길,00043,00000,11110,01,0,4100285,...,청운현대,7,9,11110-4,84.51,56-45,11110,2,,
1,"125,000",2004,2008,사직로8길,00034,00000,11110,05,0,4100135,...,경희궁의아침3단지,7,31,11110-116,124.17,72,11110,11,,
2,"72,000",2008,2008,삼봉로,00095,00000,11110,04,0,3100006,...,대성스카이렉스,7,3,11110-2223,116.03,110,11110,7,,
3,"100,000",2008,2008,삼봉로,00095,00000,11110,04,0,3100006,...,대성스카이렉스,7,10,11110-2223,149.8,110,11110,10,,
4,"89,120",2008,2008,삼봉로,00095,00000,11110,04,0,3100006,...,대성스카이렉스,7,16,11110-2223,137.55,110,11110,15,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332,"5,150",2003,2008,천제연로,00239,00000,50130,01,0,3350258,...,중문푸른마을,7,31,50130-61,51.88,1789,50130,4,,
333,"5,725",2003,2008,천제연로,00239,00000,50130,01,0,3350258,...,중문푸른마을,7,31,50130-61,59.99,1789,50130,5,,
334,"4,825",2003,2008,천제연로,00239,00000,50130,01,0,3350258,...,중문푸른마을,7,31,50130-61,51.88,1789,50130,4,,
335,"4,300",2002,2008,지세못로,00029,00000,50130,01,0,3350255,...,서진,7,11,50130-67,49.8737,367-1,50130,4,,


In [11]:
data2

,거래금액,건축년도,년,도로명,도로명건물본번호코드,도로명건물부번호코드,도로명시군구코드,도로명일련번호코드,도로명지상지하코드,도로명코드,...,아파트,월,일,일련번호,전용면적,지번,지역코드,층,해제사유발생일,해제여부
0,"50,000",2008,2008,사직로8길,00004,00000,11110,03,0,4100135,...,광화문풍림스페이스본(101동~105동),8,20,11110-2203,70.8,9,11110,12,,
1,"145,000",2008,2008,경희궁길,00057,00000,11110,01,0,4100010,...,광화문풍림스페이스본(106동),8,20,11110-2204,153.42,9-1,11110,8,,
2,"71,800",2008,2008,사직로8길,00004,00000,11110,03,0,4100135,...,광화문풍림스페이스본(101동~105동),8,25,11110-2203,94.51,9,11110,10,,
3,"110,000",2008,2008,사직로8길,00004,00000,11110,03,0,4100135,...,광화문풍림스페이스본(101동~105동),8,25,11110-2203,108.55,9,11110,14,,
4,"54,000",2008,2008,사직로8길,00004,00000,11110,03,0,4100135,...,광화문풍림스페이스본(101동~105동),8,25,11110-2203,70.8,9,11110,3,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,"4,600",2002,2008,지세못로,00029,00000,50130,01,0,3350255,...,서진,8,1,50130-67,49.8737,367-1,50130,3,,
174,"4,200",2002,2008,지세못로,00029,00000,50130,01,0,3350255,...,서진,8,4,50130-67,59.4853,367-1,50130,5,,
175,"4,400",2002,2008,지세못로,00029,00000,50130,01,0,3350255,...,서진,8,7,50130-67,49.8737,367-1,50130,2,,
176,"3,600",2000,2008,비행장로55번길,00016,00000,50130,01,0,4850299,...,한빛테크,8,8,50130-35,58.944,4148-1,50130,3,,


In [12]:
data3

,거래금액,건축년도,년,도로명,도로명건물본번호코드,도로명건물부번호코드,도로명시군구코드,도로명일련번호코드,도로명지상지하코드,도로명코드,...,아파트,월,일,일련번호,전용면적,지번,지역코드,층,해제사유발생일,해제여부
0,"68,000",2008,2008,사직로8길,00004,00000,11110,03,0,4100135,...,광화문풍림스페이스본(101동~105동),9,5,11110-2203,94.51,9,11110,9,,
1,"100,540",2008,2008,경희궁길,00057,00000,11110,01,0,4100010,...,광화문풍림스페이스본(106동),9,10,11110-2204,144.52,9-1,11110,13,,
2,"92,000",2008,2008,사직로8길,00004,00000,11110,03,0,4100135,...,광화문풍림스페이스본(101동~105동),9,22,11110-2203,151.81,9,11110,1,,
3,"95,000",2008,2008,사직로8길,00004,00000,11110,03,0,4100135,...,광화문풍림스페이스본(101동~105동),9,26,11110-2203,146.92,9,11110,11,,
4,"13,900",2003,2008,돈화문로11가길,00059,00000,11110,03,0,4100050,...,현대뜨레비앙,9,6,11110-102,36.08,55,11110,6,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,"7,400",2003,2008,천제연로,00239,00000,50130,01,0,3350258,...,중문푸른마을,9,25,50130-61,59.97,1789,50130,3,,
37,"4,750",2003,2008,천제연로,00239,00000,50130,01,0,3350258,...,중문푸른마을,9,26,50130-61,51.88,1789,50130,5,,
38,"7,200",2003,2008,천제연로,00239,00000,50130,01,0,3350258,...,중문푸른마을,9,26,50130-61,59.99,1789,50130,4,,
39,"4,250",2001,2008,표선동서로61번길,00124,00000,50130,01,0,4851020,...,표선2차반석타운,9,10,50130-82,59.58,1534-1,50130,3,,
